In [1]:
import os
import sys

sys.path.append("../../common/src")
sys.path.append("../src")

In [2]:
!export PROJECT_ID="lramsey-dev"
project = "lramsey-dev"
os.environ["PROJECT_ID"] = project
os.environ["PG_HOST"] = ""

In [3]:
from services.query.vector_store import LangChainVectorStore

INFO: [config/config.py:61 - <module>()] Namespace File not found, setting job namespace as default


/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:173: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(
/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:343: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatMlflow`.
  warn_de

INFO: [config/config.py:112 - <module>()] ENABLE_GOOGLE_LLM = True
INFO: [config/config.py:113 - <module>()] ENABLE_OPENAI_LLM = True
INFO: [config/config.py:114 - <module>()] ENABLE_COHERE_LLM = True
INFO: [config/config.py:115 - <module>()] ENABLE_GOOGLE_MODEL_GARDEN = True
INFO: [config/config.py:116 - <module>()] ENABLE_TRUSS_LLAMA2 = False
INFO: [config/config.py:117 - <module>()] ENABLE_VLLM_GEMMA = True
INFO: [config/vector_store_config.py:40 - <module>()] Default vector store = [matching_engine]
INFO: [config/vector_store_config.py:49 - <module>()] PG_HOST = [None]
INFO: [config/vector_store_config.py:50 - <module>()] PG_DBNAME = [pgvector]
INFO: [config/vector_store_config.py:79 - <module>()] PG_HOST is set to [None], not connecting to pgvector
INFO: [config/onedrive_config.py:30 - <module>()] ONEDRIVE_CLIENT_ID = [None]
INFO: [config/onedrive_config.py:31 - <module>()] ONEDRIVE_TENANT_ID = [None]


In [4]:
from common.models import (UserQuery, QueryResult, QueryEngine, QueryDocument,
                           QueryReference, QueryDocumentChunk, BatchJobModel)

In [5]:
qe_list = QueryEngine.fetch_all()
[(qe.name, qe.id) for qe in qe_list]

[('Test LLM Service 2', 'JmKH0Rnt6khHCaRfuSG7'),
 ('Test LLM Service 1', 'SaLOGu4JCvStmK7RFCMD'),
 ('Test Vertex Search 8', 'urjshigECiGqhj90MjJx'),
 ('Test Vertex Search 7', 'BY5hPEunjRYutJVD7DQa'),
 ('Test Vertex Search 6', 'vnoCpT38aKjEZlDjmhVs'),
 ('Test Vertex Search 5', 'WRDCFzLMqJFy8cvDh2WA'),
 ('APG Provider Manual', 'TXfYVDTqotmlF3thAx0O'),
 ('CHCANYS Conference web page from NYSDOH site', 'ZCfGNtk9WuIxgp8wLc6h'),
 ('FQHC Dental Policies from NYSDOH web site', 'x6UkpTjcgVZxCB8kucqU'),
 ('Medicaid dental billing policies', 'dwQL5Ir0iXDpMW4kUEM4')]

In [6]:
q_engine = QueryEngine.find_by_name('FQHC Dental Policies from NYSDOH web site')

In [7]:
langchain_vector_store = LangChainVectorStore(q_engine)

In [8]:
# Example usage
expression = 'status: ANY ("ACTIVE", "INACTIVE") AND score: IN (10, 20) OR price >= 100'
parsed_result = langchain_vector_store.parse_filter(expression)
parsed_result

AttributeError: 'Forward' object has no attribute 'expandtabs'

In [ ]:
test_filters = [
    'category:ANY("News", "Sports") AND score:IN(*, 90.5e)',
    'NOT (category:ANY("Opinion") OR score < 50)',
    'score >= 80i AND - category:ANY("Politics")'
]

for filter_str in test_filters:
    result = langchain_vector_store.parse_filter(filter_str)
    print(f"Parsed filter: {result}")